In [1]:
# import sys
# sys.path.insert(0, '..')
from semantic_mpc_interface import (
    LoadModel,
    get_thermostat_data,
    HPFlexSurvey,
    convert_units,
    SHACLHandler
)
from buildingmotif.namespaces import BRICK, RDF
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Library
import csv
from pyshacl.rdfutil import clone

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


# Testing the Model Builder

ModelBuilder has been deprecated in favor of just survey based workflow. May be readded in the future

# Testing the Survey Workflow

In [2]:
# Please disregard excessive outputs (logging and warnings)
# TODO: Figure out how to configure these in building motif 
import logging
logging.disable(logging.CRITICAL)
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Creating survey, allow overwrite if there is something already there
s = HPFlexSurvey('test_site','test_build','.', overwrite=True)

# Generating a simple building structure that prefills csv files. 
s.easy_config(zone_space_window_list=[(2,2),(1,2),(1,3)])
# Now check test_site/test_build to look at survey files


removing dependency:  space
removing dependency:  window
removing dependency:  hp-rtu
removing dependency:  tstat
values: dict_values([])
values names: {}
values: dict_values(['area-name-value'])
values names: {'area-name': '<name>-area-name'}
values: dict_values(['cooling_COP-name-value', 'heating_capacity-name-value', 'heating_COP-name-value', 'cooling_capacity-name-value'])
values names: {'heating_COP-name': '<name>-heating_COP-name', 'cooling_capacity-name': '<name>-cooling_capacity-name', 'cooling_COP-name': '<name>-cooling_COP-name', 'heating_capacity-name': '<name>-heating_capacity-name'}
values: dict_values(['resolution-name-value', 'setpoint_deadband-name-value', 'active-name-value', 'stage_count-name-value', 'tolerance-name-value'])
values names: {'active-name': '<name>-active-name', 'stage_count-name': '<name>-stage_count-name', 'resolution-name': '<name>-resolution-name', 'tolerance-name': '<name>-tolerance-name', 'setpoint_deadband-name': '<name>-setpoint_deadband-name'}
v

In [4]:
# Will just fill the columns programmatically for testing, csv should be filled out otherwise
import sys
sys.path.insert(0,'../examples')
from example_prefill_usage import prefill_csv_survey

In [5]:
prefill_csv_survey('test_site/test_build')

Loading config from: test_site/test_build/config.json
Found CSV files: ['zone', 'site', 'window', 'hvac', 'point_list', 'tstat', 'space']
Filled 10 empty cells in zone.csv
Filled 4 empty cells across 4 columns in site.csv
Filled 21 empty cells across 3 columns in window.csv
Filled 12 empty cells across 4 columns in hvac.csv
No empty cells found in point_list.csv
Filled 15 empty cells across 5 columns in tstat.csv
Filled 4 empty cells across 1 columns in space.csv


In [6]:
# Reading csv
s.read_csv()

Expanded CSV shape: (6, 5)
New columns added: []
Expanded CSV shape: (4, 4)
New columns added: ['area-name']
Expanded CSV shape: (3, 13)
New columns added: ['heating_COP-name', 'cooling_capacity-name', 'cooling_COP-name', 'heating_capacity-name']
Expanded CSV shape: (3, 14)
New columns added: ['active-name', 'stage_count-name', 'resolution-name', 'tolerance-name', 'setpoint_deadband-name']
Expanded CSV shape: (7, 10)
New columns added: ['area-name', 'tilt-name', 'azimuth-name']
Expanded CSV shape: (1, 9)
New columns added: ['noaastation-name', 'latitude-name', 'longitude-name', 'timezone-name']


# Testing SHACL Generation and Validation

In [7]:
og = clone.clone_graph(s.graph)

In [8]:
# Create handler
handler = SHACLHandler(ontology='brick')

# Generate shapes
handler.generate_shapes()

# Save shapes
handler.save_shapes('shapes.ttl')

# Validate a model
conforms, results_graph, results_text = handler.validate_model(s.graph)

s.graph.serialize('test-brick-model-reasoned.ttl', format = 'ttl')
if not conforms:
    print("Validation failed:")
    print(results_text)

In [9]:
# lots of new inferred information
(s.graph-og).print()

@prefix ns1: <urn:hpflex/shapes#> .

<urn:hpflex/test_site#hvac_1> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/test_site#zone_1> .

<urn:hpflex/test_site#hvac_2> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/test_site#zone_2> .

<urn:hpflex/test_site#hvac_3> a ns1:hp-rtu ;
    ns1:air-connected-to <urn:hpflex/test_site#zone_3> .

<urn:hpflex/test_site#test_site> a ns1:site .

<urn:hpflex/test_site#tstat_zone_1> a ns1:tstat ;
    ns1:has-location <urn:hpflex/test_site#zone_1> ;
    ns1:has-point <urn:hpflex/test_site#tstat_zone_1-active-name>,
        <urn:hpflex/test_site#tstat_zone_1-setpoint_deadband-name>,
        <urn:hpflex/test_site#tstat_zone_1-tolerance-name> .

<urn:hpflex/test_site#tstat_zone_2> a ns1:tstat ;
    ns1:has-location <urn:hpflex/test_site#zone_2> ;
    ns1:has-point <urn:hpflex/test_site#tstat_zone_2-active-name>,
        <urn:hpflex/test_site#tstat_zone_2-setpoint_deadband-name>,
        <urn:hpflex/test_site#tstat_zone_2-tolerance-name> .

<urn:hp

# Testing get Metadata

In [10]:
# still working on loader, will clean up class, but functionality about right
loader = LoadModel("test_site/test_build/test_build.ttl", ontology = 'brick')
site_info = loader.get_all_building_objects()

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.or

In [11]:
print(site_info)

{'sites': [Site(name='urn:hpflex/test_site#test_site', longitudes=1, noaastations=1, timezones=1, latitudes=1)], 'zones': [Hvac-zone(name='urn:hpflex/test_site#zone_3', tstats=1, windows=1, hvacs=1, spaces=1), Hvac-zone(name='urn:hpflex/test_site#zone_2', tstats=1, windows=1, hvacs=1, spaces=1), Hvac-zone(name='urn:hpflex/test_site#zone_1', tstats=1, windows=2, hvacs=1, spaces=2)]}


In [12]:
zone = site_info['zones'][0]
print(zone)

Hvac-zone(name='urn:hpflex/test_site#zone_3', tstats=1, windows=1, hvacs=1, spaces=1)


In [13]:
zone.windows

[Window(name='urn:hpflex/test_site#window_3_3', tilt=Value(value=7.0, unit='http://qudt.org/vocab/unit/Degree'), area=Value(value=7.0, unit='http://qudt.org/vocab/unit/FT2'), azimuth=Value(value=7.0, unit='http://qudt.org/vocab/unit/Degree'))]

In [14]:
zone.windows[0].area.name

rdflib.term.URIRef('urn:hpflex/test_site#window_3_3-area-name')

In [15]:
zone.tstats[0]

Tstat(name='urn:hpflex/test_site#tstat_zone_3', stage_count=Value(value=3.0, unit='None'), setpoint_deadband=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), active=Value(value=3.0, unit='None'), resolution=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'), tolerance=Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F'))

In [16]:
print(zone.tstats[0].resolution)
zone.tstats[0].resolution.convert_to_si()
print(zone.tstats[0].resolution)
print(zone.tstats[0].resolution.is_delta)

Value(value=3.0, unit='http://qudt.org/vocab/unit/DEG_F')
Value(value=1.6666666666666667, unit='http://qudt.org/vocab/unit/DEG_C')
True


In [17]:
# optionally just load everything as si 
si_loader = LoadModel("test_site/test_build/test_build.ttl", ontology = 'brick', as_si_units=True)
site_info = si_loader.get_all_building_objects()
print(zone.tstats[0].resolution)

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.or

In [18]:
# Getting the thermostat metadata
get_thermostat_data(si_loader)

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.or

{'heat_availability': [True, True, True],
 'cool_availability': [True, True, True],
 'heat_tolerance': [-1.6666666666666667,
  -1.1111111111111112,
  -0.5555555555555556],
 'cool_tolerance': [1.6666666666666667,
  1.1111111111111112,
  0.5555555555555556],
 'setpoint_deadband': [1.6666666666666667,
  1.1111111111111112,
  0.5555555555555556],
 'active': [],
 'control_group': ['DEPRECATED', 'DEPRECATED', 'DEPRECATED'],
 'control_type_list': [],
 'floor_area_list': [0.37161216, 0.27870912000000003, 0.27870912000000003],
 'floor_area_unit': ['http://qudt.org/vocab/unit/M2',
  'http://qudt.org/vocab/unit/M2',
  'http://qudt.org/vocab/unit/M2'],
 'window_area_list': [0.65032128, 0.37161216, 0.18580608],
 'window_area_unit': ['http://qudt.org/vocab/unit/M2',
  'http://qudt.org/vocab/unit/M2',
  'http://qudt.org/vocab/unit/M2'],
 'azimuth_list': [7.0, 4.0, 2.0],
 'azimuth_unit': ['http://qudt.org/vocab/unit/Degree',
  'http://qudt.org/vocab/unit/Degree',
  'http://qudt.org/vocab/unit/Degree']

In [19]:
# Getting the thermostat metadata for 1 zone 
get_thermostat_data(si_loader, ['zone_1','zone_2'])

PREFIX brick: <https://brickschema.org/schema/Brick#>
PREFIX csvw: <http://www.w3.org/ns/csvw#>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX dcat: <http://www.w3.org/ns/dcat#>
PREFIX dcmitype: <http://purl.org/dc/dcmitype/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dcam: <http://purl.org/dc/dcam/>
PREFIX doap: <http://usefulinc.com/ns/doap#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX odrl: <http://www.w3.org/ns/odrl/2/>
PREFIX org: <http://www.w3.org/ns/org#>
PREFIX prof: <http://www.w3.org/ns/dx/prof/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX schema: <https://schema.org/>
PREFIX sh: <http://www.w3.org/ns/shacl#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX ssn: <http://www.w3.org/ns/ssn/>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX vann: <http://purl.org/vocab/vann/>
PREFIX void: <http://rdfs.or

{'heat_availability': [True, True],
 'cool_availability': [True, True],
 'heat_tolerance': [-1.1111111111111112, -0.5555555555555556],
 'cool_tolerance': [1.1111111111111112, 0.5555555555555556],
 'setpoint_deadband': [1.1111111111111112, 0.5555555555555556],
 'active': [],
 'control_group': ['DEPRECATED', 'DEPRECATED'],
 'control_type_list': [],
 'floor_area_list': [0.27870912000000003, 0.27870912000000003],
 'floor_area_unit': ['http://qudt.org/vocab/unit/M2',
  'http://qudt.org/vocab/unit/M2'],
 'window_area_list': [0.37161216, 0.18580608],
 'window_area_unit': ['http://qudt.org/vocab/unit/M2',
  'http://qudt.org/vocab/unit/M2'],
 'azimuth_list': [4.0, 2.0],
 'azimuth_unit': ['http://qudt.org/vocab/unit/Degree',
  'http://qudt.org/vocab/unit/Degree'],
 'tilt_list': [4.0, 2.0],
 'tilt_unit': ['http://qudt.org/vocab/unit/Degree',
  'http://qudt.org/vocab/unit/Degree'],
 'zone_ids': ['zone_2', 'zone_1'],
 'hvacs': [rdflib.term.URIRef('urn:hpflex/test_site#hvac_2'),
  rdflib.term.URIRef

# Testing Unit Conversion

In [20]:
convert_units(10, 'FT', 'M')

3.048

In [21]:
convert_units(0, 'DEG_C', 'DEG_F')

31.999999999999886

In [22]:
convert_units(0, 'DEG_C', 'K')

273.15

In [23]:
# Planning to implement converting everything to si in loader
# loader.get_all_building_objects

# Testing Grafana Dashboarding 
Needs updates

In [24]:
# import yaml 
# with open('../development_files/creds.yml') as f:
#     config = yaml.safe_load(f)

# bg = BrickToGrafana(grafana_server=config['grafana_server'], grafana_api_key = config['grafana_api_key'], datasource=config['datasource'], ttl_path = 'test-brick-model.ttl')

In [25]:
# bg.create_dashboard('AQL from brick')

In [26]:
# Grafana not set up, upload won't work
# bg.upload_dashboard(message = 'testing upload')